In [3]:
!pip install transformers datasets sentencepiece --quiet
import torch
from transformers import PreTrainedTokenizerFast
from datasets import Dataset, DatasetDict
import pandas as pd
import numpy as np
from evaluate import load


In [13]:
import math

In [4]:
!pip install wandb
!pip install tensorboard
!pip install --upgrade torch torchvision --index-url https://download.pytorch.org/whl/cu118
!pip install transformers==4.29.2


In [5]:
!pip install safetensors

In [6]:
!pip install accelerate
!pip install datasets
!pip install bert-score
!pip install blobfile tiktoken
!pip install torch==2.3.1 --index-url https://download.pytorch.org/whl/cu121
!pip install datasets==3.6.0
!pip install tokenizers==0.14.1 
!pip install huggingface_hub==0.32.0
!pip install sentencepiece

In [7]:
def compute_metrics(eval_preds):
    pred_ids = eval_preds.predictions
    label_ids = eval_preds.label_ids

    # padding 토큰 처리
    pred_ids = np.where(pred_ids != -100, pred_ids, tokenizer.pad_token_id)
    label_ids = np.where(label_ids != -100, label_ids, tokenizer.pad_token_id)

    # 디코딩
    preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    refs = tokenizer.batch_decode(label_ids, skip_special_tokens=True)

    preds = [p.strip() for p in preds]
    refs = [r.strip() for r in refs]

    preds_nonempty = [p for p in preds if p]
    refs_nonempty = [r for r in refs if r]

    if not preds_nonempty or not refs_nonempty:
        return {
            "MATTR": 0.0,
            "KoBERTScore_F1": 0.0,
            "Perplexity": float("inf"),
            "Levenshtein_Distance": float("inf")
        }

    # MATTR
    def mattr(texts, window_size=50):
        def single(text):
            tokens = text.split()
            if len(tokens) < window_size:
                return len(set(tokens)) / len(tokens) if tokens else 0.0
            return np.mean([
                len(set(tokens[i:i+window_size])) / window_size
                for i in range(len(tokens) - window_size + 1)
            ])
        return np.mean([single(t) for t in texts])

    mattr_score = mattr(preds_nonempty)

    # BERTScore (KoBERT)
    bs_metric = load("bertscore", keep_in_memory=True)
    bs_results = bs_metric.compute(predictions=preds_nonempty, references=refs_nonempty, lang="ko")
    f1_kobert = float(np.mean(bs_results["f1"]))

    # Levenshtein Distance
    import editdistance
    ld = np.mean([editdistance.eval(p, r) for p, r in zip(preds_nonempty, refs_nonempty)])

    # PPL: preds + refs 기반 계산
    def compute_perplexity_from_preds(preds, refs):
        model.eval()
        total_logp = 0.0
        total_tokens = 0

        for pred_text, ref_text in zip(preds, refs):
            input_ids = tokenizer(pred_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            labels = tokenizer(ref_text, return_tensors="pt", padding=True, truncation=True).input_ids.to(model.device)
            labels[labels == tokenizer.pad_token_id] = -100
            with torch.no_grad():
                outputs = model(input_ids=input_ids, labels=labels)
                loss = outputs.loss
            n_tokens = (labels != -100).sum().item()
            total_logp += loss.item() * n_tokens
            total_tokens += n_tokens

        return math.exp(total_logp / total_tokens) if total_tokens > 0 else float("inf")

    ppl = compute_perplexity_from_preds(preds_nonempty, refs_nonempty)

    return {
        "MATTR": round(mattr_score, 4),
        "KoBERTScore_F1": round(f1_kobert, 4),
        "Perplexity": round(ppl, 4),
        "Levenshtein_Distance": round(ld, 2),
    }

In [10]:

import pandas as pd
import torch
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, Trainer, TrainingArguments, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset, DatasetDict

model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model     = BartForConditionalGeneration.from_pretrained(model_name)
df = pd.read_csv("Data_filtered.csv", encoding="utf-8")
df = df[["ko_translationese", "ko"]].rename(columns={"ko_translationese": "input_text", "ko": "target_text"})
df = df.dropna()
df = df[df["input_text"].str.strip() != ""]
df = df[df["target_text"].str.strip() != ""]

dataset = Dataset.from_pandas(df)
dataset = dataset.train_test_split(test_size=0.1, seed=42)

def preprocess_function(example):
    inputs = tokenizer(example["input_text"], max_length=128, padding="max_length", truncation=True)
    targets = tokenizer(example["target_text"], max_length=128, padding="max_length", truncation=True)
    inputs["labels"] = targets["input_ids"]
    return inputs

tokenized = dataset.map(preprocess_function, remove_columns=dataset["train"].column_names)

from transformers import Seq2SeqTrainingArguments
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq

model_name = "gogamza/kobart-base-v2"
tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name)
model     = BartForConditionalGeneration.from_pretrained(model_name)

from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir="./kobart_translationese",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="steps",
    logging_steps=100,
    learning_rate=3e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    num_train_epochs=5,
    weight_decay=0.01,

    predict_with_generate=True,       # Seq2Seq 전용
    generation_max_length=64,         # Seq2Seq 전용

    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    fp16=True,
    report_to=["wandb", "tensorboard"],
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [11]:
# !pip install transformers==4.52.4


In [14]:
trainer.train()

model.save_pretrained("./kobart_translationese_final")


Epoch,Training Loss,Validation Loss,Mattr,Kobertscore F1,Perplexity,Levenshtein Distance
1,0.170700,0.186114,0.993500,0.866400,8.341800,14.140000
2,0.144600,0.185334,0.993200,0.866200,8.836600,14.250000
3,0.125300,0.187664,0.993600,0.866300,9.501400,14.200000
4,0.113500,0.190936,0.993400,0.865700,10.158700,14.300000
5,0.110500,0.190937,0.993400,0.865700,10.158600,14.300000


In [18]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_path = "kobart_translationese/checkpoint-12037"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to("cuda")
model.eval()

input_text = "UV 보호 기능을 가지고 있습니다. 400mm까지의 모든 유해한 파란색 빛을 100% 필터링합니다. 하지만 착용하기에 매우 편리합니다."

inputs = tokenizer(
    input_text,
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=128,
    return_token_type_ids=False    
).to("cuda")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        decoder_start_token_id=tokenizer.eos_token_id,
        max_new_tokens=256,
        num_beams=5
    )

result = tokenizer.decode(outputs[0], skip_special_tokens=True).strip()
print("✅ 입력:", input_text)
print("✅ 출력:", result)


✅ 입력: UV 보호 기능을 가지고 있습니다. 400mm까지의 모든 유해한 파란색 빛을 100% 필터링합니다. 하지만 착용하기에 매우 편리합니다.
✅ 출력: 최대 400mm의 유해 블루라이트 빛을 100% 필터링하여 착용감이 매우 우수합니다.


In [21]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch

model_dir = "kobart_translationese/checkpoint-12037"
tokenizer = AutoTokenizer.from_pretrained(model_dir)
model = AutoModelForSeq2SeqLM.from_pretrained(model_dir).to("cuda")
model.eval()

translationese_list = [
    '철수는 10개의 소를 키운다.',
    '그 아이들은 보육원에 의해 보호되었다.',
    '전통적인 중국 문화에서 결혼에 대한 결정들은 그들의 자녀를 위하는 부모들에 의해 만들어졌다.',
    '다음 물음에 대해 답하시오.',
    '고모는 두 명을 아들을 가지고 있다.',
    '나는 미국 여행을 가기 위해 영어를 공부했다.',
    '학생들은 공부함에 있어서 불편함이 없었다.',
    '성공하기 위해서는 열심히 노력하지 않으면 안 된다.',
    '그들 대부분은 좋은 직장을 얻는 데는 새로운 아이디어들과 방법들이 있다는 것을 모른다.',
    '두 사람이 주점의 2층에서의 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었다.'
]

for idx, sentence in enumerate(translationese_list, 1):
    inputs = tokenizer(
        sentence,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=128
    ).to("cuda")

    # token_type_ids 키가 있으면 제거
    inputs.pop("token_type_ids", None)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            decoder_start_token_id=tokenizer.eos_token_id,
            max_new_tokens=128,
            num_beams=5,
            early_stopping=True
        )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "<extra_id_0>" in decoded:
        result = decoded.split("<extra_id_0>", 1)[-1].strip()
    else:
        result = decoded.strip()

    # 문장 앞의 불필요한 공백/특수문자 제거
    result = result.lstrip(" .>")

    print(f"입력 {idx}: {sentence}")
    print(f"완화 {idx}: {result}\n")


입력 1: 철수는 10개의 소를 키운다.
완화 1: 철수는 10마리의 소를 키웁니다.

입력 2: 그 아이들은 보육원에 의해 보호되었다.
완화 2: 그 아이들은 보육원에서 보호되었어요.

입력 3: 전통적인 중국 문화에서 결혼에 대한 결정들은 그들의 자녀를 위하는 부모들에 의해 만들어졌다.
완화 3: 전통적인 중국 문화에서 결혼은 자녀를 위하는 부모들에 의해 결정됩니다.

입력 4: 다음 물음에 대해 답하시오.
완화 4: 다음 질문에 답해 주시기 바랍니다.

입력 5: 고모는 두 명을 아들을 가지고 있다.
완화 5: 고모는 두 아들을 두었어요.

입력 6: 나는 미국 여행을 가기 위해 영어를 공부했다.
완화 6: 미국 여행을 가려고 영어를 공부하고 있어요.

입력 7: 학생들은 공부함에 있어서 불편함이 없었다.
완화 7: 학생들은 공부하는 데 불편함이 없습니다.

입력 8: 성공하기 위해서는 열심히 노력하지 않으면 안 된다.
완화 8: 성공을 위해서는 열심히 노력하지 않으면 안 됩니다.

입력 9: 그들 대부분은 좋은 직장을 얻는 데는 새로운 아이디어들과 방법들이 있다는 것을 모른다.
완화 9: 대부분은 좋은 직장을 얻는 데 새로운 아이디어와 방법이 있다는 것을 모른다.

입력 10: 두 사람이 주점의 2층에서의 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었다.
완화 10: 2층에 살았던 살림을 그만두고 교외로 이사하여 노라의 아기 주디와 살고 있다는 이야기를 들었습니다.

